In [6]:
#Libs do qdrant
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

In [7]:
#Libs do Langchain
from langchain.vectorstores import Qdrant
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.docstore.document import Document
import pandas as pd
from pprint import pprint

In [9]:
from dotenv import load_dotenv


load_dotenv()

client = QdrantClient(host="localhost", port=6333)
client.create_collection(
    collection_name="descomplicando_qdrant_tickets",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

True

In [10]:
embeddings = OpenAIEmbeddings()

vectorstore = Qdrant(
        client=client,
        collection_name="descomplicando_qdrant_tickets",
        embeddings=embeddings
    )

/tmp/ipykernel_9792/985381723.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()
/tmp/ipykernel_9792/985381723.py:3: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  vectorstore = Qdrant(


In [20]:
def quebrar_csv_helpdesk(caminho_arquivo, tamanho_chunk=1000, sobreposicao=200):

    df = pd.read_csv(caminho_arquivo, sep=',')
    

    colunas_csv = df.columns.tolist()
    print("Nomes das colunas no CSV:", colunas_csv)
    

    colunas_texto = colunas_csv
    colunas_metadata = {
        "Ticket ID": "Integer",
        "Customer Name": "String",
        "Customer Email": "String",
        "Customer Age": "Integer",
        "Customer Gender": "String",
        "Product Purchased": "String",
        "Date of Purchase": "String",
        "Ticket Type": "String",
        "Ticket Subject": "String",
        "Ticket Description": "String",
        "Ticket Status": "String",
        "Resolution": "String",
        "Ticket Priority": "String",
        "Ticket Channel": "String",
        "First Response Time": "String",
        "Time to Resolution": "String",
        "Customer Satisfaction Rating": "Float"
    }

    documentos = []
    for linha, row in df.iterrows():
        # Criar o texto completo com as colunas corretas
        texto_completo = "\n\n".join([f"{col}: {row[col]}" for col in colunas_texto if pd.notna(row[col])])
        print("Tamanho total do Texto: ",len(texto_completo)," para a linha: ", linha)
        
  
        metadata = {col: row[col] for col in colunas_metadata if pd.notna(row[col])}
        
        
        if texto_completo.strip():
            doc = Document(page_content=texto_completo, metadata=metadata)
            documentos.append(doc)
            print("Adicionado a collection o documento da linha: ",linha)
    
    print("Finalizado.")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=tamanho_chunk,
        chunk_overlap=sobreposicao,
        length_function=len,
        separators=["\n\n", "\n", ";", " ", ""]
    )
    
    
    chunks = text_splitter.split_documents(documentos)
    return chunks

In [21]:
texto_chunks = quebrar_csv_helpdesk("../customer_support_tickets.csv")
vectorstore.add_documents(texto_chunks)

Nomes das colunas no CSV: ['Ticket ID', 'Customer Name', 'Customer Email', 'Customer Age', 'Customer Gender', 'Product Purchased', 'Date of Purchase', 'Ticket Type', 'Ticket Subject', 'Ticket Description', 'Ticket Status', 'Resolution', 'Ticket Priority', 'Ticket Channel', 'First Response Time', 'Time to Resolution', 'Customer Satisfaction Rating']
Tamanho total do Texto:  697  para a linha:  0
Adicionado a collection o documento da linha:  0
Tamanho total do Texto:  697  para a linha:  1
Adicionado a collection o documento da linha:  1
Tamanho total do Texto:  803  para a linha:  2
Adicionado a collection o documento da linha:  2
Tamanho total do Texto:  795  para a linha:  3
Adicionado a collection o documento da linha:  3
Tamanho total do Texto:  836  para a linha:  4
Adicionado a collection o documento da linha:  4
Tamanho total do Texto:  640  para a linha:  5
Adicionado a collection o documento da linha:  5
Tamanho total do Texto:  700  para a linha:  6
Adicionado a collection o 

['8526004a8ffc4735804673abf2907081',
 'bc6b62ad60a0412cbe8f1f1dc52e5c7e',
 '3f87dede110447ff891145953988e4c9',
 'e8d80c3b6b44468f99e3fa4f90eaac73',
 'c217cc0505a3463fbe90b054b29e06c3',
 'cd23bcaca0fe4c9a8aef2b355716ac25',
 'efeaa67ca2d241eab0aa4485ac49f1b8',
 '98a885c994304816b6aaeaaef02f2e9a',
 '1469feae7ea24bfca7855c3bee2a2990',
 'b65c15db4d704fcaa99b64ac2662192b',
 '4493b979372f47fd9d209784fdab9832',
 '805471f10a154cf09f844c97564e4b18',
 '746190de995a431192e653fdd1366de3',
 '53a1086246684214ba761dffcadba637',
 '233dc05f72584db0a4205a5e63040270',
 '7756ddb001964afabddd98e41a65f5cf',
 '2408cd1f258e4ab4a11c52abe5fb0eee',
 '9a634a87807145e9a55d14afd2d3b095',
 '12e8ecba4c824f5f93b8d5e4b8fd8acb',
 'c2db25234b964440960e7a8e384d8b23',
 '0d122fe6f1504c4093f6b9af05549cde',
 '21c27cc4039e4e40974eb18fe7aa00e1',
 '1c8b7eac002141d99561476c87fb6ff1',
 '492eb6a967c9434588611a02bf921d9b',
 '02b8010fcc614045abfbff8789c29060',
 '235bea9d93694fe2bb393718a8d050c5',
 'da7c91904a124f56be42d9631dea35f2',
 

In [22]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
    return_source_documents=True
)

/tmp/ipykernel_9792/2660574750.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


In [23]:
def qa_system(pergunta):
    resultado = qa({"query": pergunta})
    return {
        "resposta": resultado["result"],
        "fontes": resultado["source_documents"]
    }

In [27]:
while True:
    pergunta = input("\nSua pergunta (ou 'sair' para encerrar): ")
    
    if pergunta.lower() == "sair":
        break
    

    resultado = qa_system(pergunta)
    
    print("\nPergunta:", pergunta)
    print("\nResposta:", resultado["resposta"])